In [1]:
!pip install pint
import pint
ureg = pint.UnitRegistry()

In [2]:
import pandas as pd

columns = ['scenario', 'mass (kg)', 'frontal_area (m^2)', 'drag_coefficient', 'tire_radius (m)', 'tire_pressure (psi)', 'velocity (km/h)']
mechanical_input_df = pd.DataFrame(columns=columns)
mechanical_input_df.index.name = 'Scenario_ID'
mechanical_input_df.loc[0] = ['Elysia', 600, 1.7, 0.14, 0.2286, 35, 60]
mechanical_input_df.loc[1] = ['Helios', 1000, 1.66, 0.135, 0.254, 35, 60]

mechanical_input_df

,scenario,mass (kg),frontal_area (m^2),drag_coefficient,tire_radius (m),tire_pressure (psi),velocity (km/h)
Scenario_ID,,,,,,,
0,Elysia,600,1.70,0.140,0.2286,35,60
1,Helios,1000,1.66,0.135,0.2540,35,60


In [3]:
number_of_rows = mechanical_input_df.shape[0]

# Car Variables 

columns = ['scenario', 'rolling_resistance_coef', 'rolling_resistance_force (N)', 'drag_force (N)', 'total_mechanical_force (N)', 'mechanical_power (kW)']
mechanical_output_df = pd.DataFrame(columns=columns)
mechanical_output_df.index.name = 'Scenario_ID'

for id in range (0, number_of_rows):

    scenario = mechanical_input_df.iloc[id]['scenario']

    mass = mechanical_input_df.iloc[id]['mass (kg)'] * ureg.kilogram
    frontal_area = mechanical_input_df.iloc[id]['frontal_area (m^2)'] * ureg.meter**2
    drag_coefficient = mechanical_input_df.iloc[id]['drag_coefficient']
    tire_radius = mechanical_input_df.iloc[id]['tire_radius (m)'] * ureg.m
    tire_pressure = mechanical_input_df.iloc[id]['tire_pressure (psi)'] * ureg.psi
    velocity = mechanical_input_df.iloc[id]['velocity (km/h)'] * ureg.kilometer / ureg.hour

    # Environment Variables

    air_density = 1.164 * ureg.kilogram / ureg.meter**3
    gravity = 9.81 * ureg.meter / ureg.second**2

    # Force Calculations 
    # DO NOT MODIFY THIS SECTION

    drag_force = (0.5 * air_density * frontal_area * drag_coefficient * velocity.to(ureg.meter / ureg.second)**2).to(ureg.newton)

    rolling_resistance_coef = 0.005 + (1 * ureg.bar  / tire_pressure.to(ureg.bar)) * (0.01 + 0.0095*pow(velocity/100, 2) * (1 * ureg.hour**2 / ureg.kilometer**2)) 
    rolling_resistance_force = (rolling_resistance_coef * mass * gravity).to(ureg.newton)

    total_mechanical_force = drag_force + rolling_resistance_force
    mechanical_power = total_mechanical_force * velocity.to(ureg.meter_per_second)
    mechanical_power = mechanical_power.to(ureg.kilowatts)

    # Force Output Data
    mechanical_output_df.loc[id] = [scenario, round(rolling_resistance_coef,4), round(rolling_resistance_force,4), round(drag_force,4), round(total_mechanical_force,4), round(mechanical_power,4)]

mechanical_output_df



/Users/billysidharta/anaconda3/envs/personal/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


,scenario,rolling_resistance_coef,rolling_resistance_force (N),drag_force (N),total_mechanical_force (N),mechanical_power (kW)
Scenario_ID,,,,,,
0,Elysia,0.0106 dimensionless,62.163 newton,38.4767 newton,100.6397 newton,1.6773 kilowatt
1,Helios,0.0106 dimensionless,103.605 newton,36.2295 newton,139.8345 newton,2.3306 kilowatt


#### Temporary Battery Pack Capacity Calculation

Theoretical steps when more information given
- use mechanical force to calculate torque applied to motor
- relate to motor rpm to achieve desired speed
- Determine efficiency

In [4]:
columns = ['motor', 'motor_efficiency', 'run_duration (hr)']
electrical_input_df = pd.DataFrame(columns=columns)
electrical_input_df.index.name = 'Scenario_ID'

electrical_input_df.loc[0] = ['Marand', 0.983, 8]
electrical_input_df.loc[1] = ['GEM', 0.8, 8]

electrical_input_df

,motor,motor_efficiency,run_duration (hr)
Scenario_ID,,,
0,Marand,0.983,8
1,GEM,0.800,8


In [5]:
columns = ['electrical_power (kW)', 'electrical_energy (kWh)']
electrical_output_df = pd.DataFrame(columns=columns)
electrical_output_df.index.name = 'Scenario_ID'

for id in range (0, number_of_rows):

    run_duration = electrical_input_df.loc[id]['run_duration (hr)'] * ureg.hour

    electrical_power = mechanical_output_df.loc[id]['mechanical_power (kW)'] * (1 /electrical_input_df.loc[id]['motor_efficiency'])
    electrical_energy = electrical_power * run_duration
    electrical_energy = electrical_energy.to(ureg.kilowatt_hour)

    electrical_output_df.loc[id] = [round(electrical_power,4), round(electrical_energy,4)]

electrical_output_df


/Users/billysidharta/anaconda3/envs/personal/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: UnitStrippedWarning: The unit of the quantity is stripped when downcasting to ndarray.
  result[:] = values


,electrical_power (kW),electrical_energy (kWh)
Scenario_ID,,
0,1.7063 kilowatt,13.6505 kilowatt_hour
1,2.9132 kilowatt,23.306 kilowatt_hour


#### Output

In [6]:
input_complete_df = pd.concat([mechanical_input_df, electrical_input_df], axis = 1)
input_complete_df

,scenario,mass (kg),frontal_area (m^2),drag_coefficient,tire_radius (m),tire_pressure (psi),velocity (km/h),motor,motor_efficiency,run_duration (hr)
Scenario_ID,,,,,,,,,,
0,Elysia,600,1.70,0.140,0.2286,35,60,Marand,0.983,8
1,Helios,1000,1.66,0.135,0.2540,35,60,GEM,0.800,8


In [7]:
output_complete_df = pd.concat([mechanical_output_df, electrical_output_df], axis = 1)
output_complete_df

,scenario,rolling_resistance_coef,rolling_resistance_force (N),drag_force (N),total_mechanical_force (N),mechanical_power (kW),electrical_power (kW),electrical_energy (kWh)
Scenario_ID,,,,,,,,
0,Elysia,0.0106 dimensionless,62.163 newton,38.4767 newton,100.6397 newton,1.6773 kilowatt,1.7063 kilowatt,13.6505 kilowatt_hour
1,Helios,0.0106 dimensionless,103.605 newton,36.2295 newton,139.8345 newton,2.3306 kilowatt,2.9132 kilowatt,23.306 kilowatt_hour
